In [1]:
import pandas as pd
import numpy as np
import imp

import data
import util

imp.reload(data)
imp.reload(util)

<module 'util' from '/Users/grahamseamans/classes/data_science/ds-project/src/util.py'>

In [59]:
daily = data.get_transit_daily()
# gets a smaller version for faster troubleshooting
daily = data.data_transforms(daily)

In [60]:
x = daily[data.USED_COLS + ["trip_id"]]

so how does this work:
* each stop gets its own row
* get a list with every stop
* isolate each trip (get a list with every trip, pull out the trip one at a time)
* for each stop, check the trip table, add that row to the image, or dont if there's nothing there
* sort them by arrive time somehow?
* maybe I get them out of the gtfs data instead...
* maybe do something else with the categorical data?

In [61]:
trip_ids = daily["trip_id"].unique().to_list()
df = x

non_categories = list(set(data.USED_COLS) - set(data.CATEGORIES))
iding_the_row = ["trip_id", "direction", "stop_id"]
for_image = df[non_categories + iding_the_row]

trip_images = []
trip_labels = []

for trip_id in trip_ids:
    trip = for_image[for_image["trip_id"] == trip_id]

    stop_id_list = []

    if trip.iloc[0]["direction"] == 0:
        stop_id_list = data.DIR_0_STOP_IDS
        #     else:
        #         stop_id_list = data.DIR_1_STOP_IDS

        image = np.zeros((len(stop_id_list), len(non_categories) + 1))
        label = np.zeros((len(stop_id_list)))

        for i, stop_id in enumerate(stop_id_list):
            row = trip[trip["stop_id"] == stop_id][
                ["label"] + non_categories
            ].to_numpy()
            if row.shape[0] > 0:
                row = row[0]
                image[i] = row
                label[i] = row[0]

        trip_images.append(image)
        trip_labels.append(label)
        np.set_printoptions(precision=1, suppress=True)

trip_images = np.stack(trip_images, axis=0)
trip_labels = np.stack(trip_labels, axis=0)

In [62]:
# x = pd.get_dummies(x)  # turns all categoricals into one hot encoded columns!

label = x.pop("label")
x = trip_images

In [63]:
print(trip_labels.shape)
print(trip_images.shape)

(73, 80)
(73, 80, 11)


In [64]:
util.big_print(trip_labels)

[[  71.   61.   58. ...  -31.  -39.    0.]
 [  86.   42.   19. ...    1.  -18.    0.]
 [ 132.  132.  129. ... -187. -181.    0.]
 ...
 [ 100.   67.  105. ... -183.    0.    0.]
 [ 200.  156.  153. ... -176.    0.    0.]
 [  71.   27.   37. ...    0.    0.    0.]]


In [6]:
print("number of input features: ", len(sorted(x)))

number of input features:  568


In [7]:
import tensorflow as tf

tf.random.set_seed(1234)

# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe

# if batch size is one the net freaks out (maybe traning, maybe testing, either way, it's bad)
batch_size = 50
data_len = len(x)
train = 0.8
val = 0.1
test = 0.1

assert train + val + test == 1

train_size = int(train * data_len)
val_size = int(val * data_len)

dataset = tf.data.Dataset.from_tensor_slices((x.values, label.values))

# only shuffles the training set
train_dataset = dataset.take(train_size).shuffle(train_size).batch(batch_size)
val_dataset = dataset.skip(train_size).take(val_size).batch(batch_size)
test_dataset = dataset.skip(train_size).skip(val_size).batch(batch_size)

KeyboardInterrupt: 

In [ ]:
def get_compiled_model():
    model = tf.keras.Sequential(
        [
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(500, activation="relu"),
            tf.keras.layers.Dense(100, activation="relu"),
            tf.keras.layers.Dense(50, activation="relu"),
            tf.keras.layers.Dense(50, activation="relu"),
            tf.keras.layers.Dense(50, activation="relu"),
            #             tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.MeanSquaredError(
            reduction="auto", name="mean_squared_error"
        ),
        metrics=["accuracy", "mean_absolute_error"],
    )

    return model

In [ ]:
model = get_compiled_model()
callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_mean_absolute_error", patience=10, restore_best_weights=True
)
model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[callback])

In [ ]:
model.evaluate(test_dataset)

In [ ]:
preds = []
labels = []
for data in test_dataset:
    preds.append(model.predict_on_batch(data[0]).flatten())
    labels.append(data[1])

preds = [p.tolist() for p in preds]
labels = [l.numpy().tolist() for l in labels]

preds = util.flatten(preds)
labels = util.flatten(labels)
assert len(preds) == len(labels)

In [ ]:
from matplotlib import pyplot as plt

# adjust these numbers to change the width and height of the plot!
plt.rcParams["figure.figsize"] = [20, 7]

util.plot(69, 420, preds, labels)

util.allDone()